In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

In [ ]:
!pip install nltk
!pip install optuna

import pandas as pd
import random
from openai import OpenAI
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.translate.meteor_score import meteor_score
from itertools import combinations
import optuna
import json
import gzip

# Ensure the necessary NLTK resources are downloaded
nltk.download('wordnet')

API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125'

In [ ]:
# Replace 'path_to_file.jsonl' with the path to your JSONL file
file_path = 'GSM8K.jsonl'

data = []

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Convert each JSON string into a Python dictionary
        json_dict = json.loads(line)
        data.append(json_dict)

# 'data' is now a list of dictionaries
print(data[11]['question'])
print( re.findall(r'\n#### (.*)', data[11]['answer'])[0] )


In [ ]:
def mean_pairwise_meteor(prompts):
    """
    Compute the mean pairwise METEOR score among a list of prompts.

    :param prompts: List of generated text outputs
    :return: Mean METEOR score across all prompt pairs
    """
    # Tokenize each prompt into a list of words
    tokenized_prompts = [p.split() for p in prompts]

    pairs = list(combinations(tokenized_prompts, 2))  # Get all unique pairs
    scores = [meteor_score([p1], p2) for p1, p2 in pairs]  # Compute METEOR for each pair

    return sum(scores) / len(scores) if scores else 0

# # Example usage
# prompts = ["this is a test sentence", "this is an example test", "another test case"]
# mean_score = mean_pairwise_meteor(prompts)
# print("Mean Pairwise METEOR Score:", mean_score)



def mean_pairwise_cosine_similarity(arrays):
    # Stack arrays into a single matrix
    matrix = np.vstack(arrays)

    # Compute cosine similarity matrix
    sim_matrix = cosine_similarity(matrix)

    # Exclude diagonal elements (self-similarity)
    n = len(arrays)
    mask = np.ones((n, n), dtype=bool)
    np.fill_diagonal(mask, 0)

    # Compute the average pairwise similarity
    avg_similarity = sim_matrix[mask].mean()

    return avg_similarity

# # Example usage
# arrays = [np.random.rand(10) for _ in range(5)]  # List of 5 random 10-dimensional vectors
# avg_sim = mean_pairwise_cosine_similarity(arrays)
# print("Mean Pairwise Cosine Similarity:", avg_sim)



def ADO(num_prompts, cos_sim, lexical_sim):

    samples = []
    targets = []
    for i in range(10):
        samples.append(data[i]['question'])
        targets.append( re.findall(r'\n#### (.*)', data[i]['answer'])[0] )

    ADO_prompt = f"""
    You are given a math question statement. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich this question statement. The goal is of the algorithm is to perform a thorough engineering on the statement, so that it is easier for a LLM to solve it. Below are some sample math question statements as refrences.

    Examples:
    - Job Description: '{samples[0]}'; Is the job Fraud or not: '{targets[0]}'
    - Job Description: '{samples[1]}'; Is the job Fraud or not: '{targets[1]}'
    - Job Description: '{samples[2]}'; Is the job Fraud or not: '{targets[2]}'
    - Job Description: '{samples[3]}'; Is the job Fraud or not: '{targets[3]}'
    - Job Description: '{samples[4]}'; Is the job Fraud or not: '{targets[4]}'
    - Job Description: '{samples[5]}'; Is the job Fraud or not: '{targets[5]}'
    - Job Description: '{samples[6]}'; Is the job Fraud or not: '{targets[6]}'
    - Job Description: '{samples[7]}'; Is the job Fraud or not: '{targets[7]}'
    - Job Description: '{samples[8]}'; Is the job Fraud or not: '{targets[8]}'
    - Job Description: '{samples[9]}'; Is the job Fraud or not: '{targets[9]}'

    Important: You MUST NOT solve the samples above! The samples are only to get you familiar to the statements you will enrich and reformulate.

    For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
    Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform vector generations. Do NOT propose to draw anything. Do NOT perform similarity checking. Do NOT propose how to train or generate a recommendation system. ONLY propose steps that a LLM can generate on its own!!!
    """

    search_rounds = 3
    tuning_size = 10
    prompt_performances = {}

    for s in range(search_rounds):

        total_try = 0
        prompts = []
        prompt_embeddings = []
        prompt_cos_dict = {}
        mean_cos_sim = 1.0

        while mean_cos_sim > cos_sim:

            for i in range(num_prompts):
                completion = client.chat.completions.create(
                    model = model_id, temperature = 1, max_tokens = 1000,

                    messages=[{"role": "system", "content": "Please reformulate and enrich the math question to be extremely informative and detailed for LLM to interpret better. You are allowed to infer and fill-in unspecified information based on your domain expertise!"},
                                {"role": "user", "content": ADO_prompt}],
                    timeout = 1200)

                candidate_prompt = completion.choices[0].message.content
                prompts.append(candidate_prompt)

                response = client.embeddings.create(
                    input=candidate_prompt,
                    model="text-embedding-3-small",
                )

                candidate_prompt_embedding = np.array(response.data[0].embedding)
                prompt_embeddings.append(candidate_prompt_embedding)

                # ADO_prompt += '\n\n\n Please generate each step to be completely different in wording and semantics from this one: \n\n' + candidate_prompt + '\n\n\n'


            total_try += 1
            mean_meteor = mean_pairwise_meteor(prompts)
            mean_cos_sim = mean_pairwise_cosine_similarity(prompt_embeddings)

            # Define min and max based on typical ranges (adjustable)
            meteor_min, meteor_max = 0.0, 1.0  # METEOR is usually in [0,1]
            cos_min, cos_max = 0.5, 1.0  # Cosine similarity often ranges [0.5,1] in text similarity tasks

            # Min-max normalization
            meteor_norm = (mean_meteor - meteor_min) / (meteor_max - meteor_min)
            cos_norm = (mean_cos_sim - cos_min) / (cos_max - cos_min)

            prompt_cos_dict[tuple(prompts)] = meteor_norm + cos_norm

            if mean_cos_sim <= cos_sim and mean_meteor <= lexical_sim:
                print('Qualifying prompts generated!')
                break

            if total_try > 5:
                prompts = min(prompt_cos_dict, key=prompt_cos_dict.get)
                break

            prompts = []
            prompt_embeddings = []


        # Now we have the candidate prompts, start tuning set evaluation:
        for candidate_prompt in prompts:

            print(candidate_prompt)
            print()

            # Regular expression to split the text into individual algorithms
            algorithm_pattern = r"###------- Step \d: [\w\s]+"

            # Split the text based on the pattern
            split_text = re.split(algorithm_pattern, candidate_prompt)

            # Extract the algorithm headers (for identification)
            headers = re.findall(algorithm_pattern, candidate_prompt)

            # Removing the first empty string from the split if exists (because of the split at the start)
            split_text = [t.strip() for t in split_text if t.strip()]

            # Create a dictionary where each algorithm is stored separately
            algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

            # Display each algorithm separately
            steps = []
            for header, content in algorithms.items():
                steps.append( [header, content] )
                # print(f"{header}:\n{content}\n")

            total_right = 0
            ADO_total_right = 0
            total = 0

            for i in range( len(data) ):
                question = data[i]['question']
                target = re.findall(r'\n#### (.*)', data[i]['answer'])[0]

                original_prompt = (
                    'Given a math question, please solve it by returning the final answer as a number.' +
                    f'\n\nQuestion: {question}' +
                    "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
                )

                completion = client.chat.completions.create(
                    model = model_id, temperature = 0, seed = 0,

                    messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                                {"role": "user", "content": original_prompt}],
                    timeout = 1200)

                ori_answer = completion.choices[0].message.content

                total += 1
                match = re.search(r'[-+]?\d*\.\d+|\d+', ori_answer)
                if match:
                    ori_answer = float(match.group())

                if float(target) == float(ori_answer):
                    total_right += 1


                overall_with_steps = "Original question statement: " + question + '\n\n'
                for i in range(len(algorithms)):

                    reformulation_prompt = f"Please thoroughly reformulate the math question statement based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nQuestion statement to reformulate: {overall_with_steps}."

                    completion = client.chat.completions.create(
                            model = model_id, temperature = 0.0, max_tokens = 768,

                            messages=[{"role": "system", "content": "Please reformulate and enrich the math question to be extremely informative and detailed for LLM to interpret better. You are allowed to infer and fill-in unspecified information based on your domain expertise!"},
                                        {"role": "user", "content": reformulation_prompt}],
                            timeout = 1200)

                    reformulated_history = completion.choices[0].message.content
                    overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'

                ADO_prompt = (
                    'Given a math question, please solve it by returning the final answer as a number.' +
                    f'\n\nQuestion: {overall_with_steps}' +
                    "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
                )

                completion = client.chat.completions.create(
                    model = model_id, temperature = 0, seed = 0,

                    messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                                {"role": "user", "content": ADO_prompt}],
                    timeout = 1200)

                answer = completion.choices[0].message.content

                match = re.search(r'[-+]?\d*\.\d+|\d+', answer)
                if match:
                    answer = float(match.group())

                if float(target) == float(answer):
                    ADO_total_right += 1

                print(target, ori_answer, answer)
                print(f'Total: {total}, Total Right: {total_right}, ADE Total Right: {ADO_total_right}')
                print('----------------------------------------------------------------------------------------')
                print()

                if total % 50 == 0:
                    if total_right >= ADO_total_right:
                        break

                if total >= tuning_size:
                    break

            prompt_performances[candidate_prompt] = ADO_total_right/total
            print( prompt_performances[candidate_prompt] )

        prior_exp = ""
        for k,v in prompt_performances.items():
            prior_exp += '\n\n' + 'Algorithm: ' + k + '\n\n' + 'Score: ' + str(v)

        ADO_prompt = f"""
        You are given a math question statement. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich this question statement. The goal is of the algorithm is to perform a thorough engineering on the statement, so that it is easier for a LLM to solve it. Below are some sample math question statements as refrences.

        Examples:
        - Job Description: '{samples[0]}'; Is the job Fraud or not: '{targets[0]}'
        - Job Description: '{samples[1]}'; Is the job Fraud or not: '{targets[1]}'
        - Job Description: '{samples[2]}'; Is the job Fraud or not: '{targets[2]}'
        - Job Description: '{samples[3]}'; Is the job Fraud or not: '{targets[3]}'
        - Job Description: '{samples[4]}'; Is the job Fraud or not: '{targets[4]}'
        - Job Description: '{samples[5]}'; Is the job Fraud or not: '{targets[5]}'
        - Job Description: '{samples[6]}'; Is the job Fraud or not: '{targets[6]}'
        - Job Description: '{samples[7]}'; Is the job Fraud or not: '{targets[7]}'
        - Job Description: '{samples[8]}'; Is the job Fraud or not: '{targets[8]}'
        - Job Description: '{samples[9]}'; Is the job Fraud or not: '{targets[9]}'

        Important: You MUST NOT solve the samples above! The samples are only to get you familiar to the statements you will enrich and reformulate.

        For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
        Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform vector generations. Do NOT propose to draw anything. Do NOT perform similarity checking. Do NOT propose how to train or generate a recommendation system. ONLY propose steps that a LLM can generate on its own!!!
        """

        ADO_prompt += '\n\nBelow are some algorithm-score pairs for you to refer to prior to generation:' + '\n' + prior_exp
        print(ADO_prompt)
        print()

    prompt = max(prompt_performances, key=prompt_performances.get)
    return [prompt, prompt_performances[prompt]]


# # example run
# output = ADO(2, 0.7, 0.3)

# print(output[0])
# print()
# print(output[1])

In [ ]:
# perform Bayes Opt.
best_prompt_per_round = {}

def objective(trial):
    num_prompts = trial.suggest_int('num of prompts', 2, 4)
    cos_sim = trial.suggest_float('Mean Cosine Sim among prompts', 0.6, 0.95, step = 0.05)
    lexical_sim = trial.suggest_float('Mean Meteor among prompts', 0.2, 0.5, step = 0.05)

    outputs = ADO(num_prompts, cos_sim, lexical_sim)
    best_prompt_per_round[ outputs[0] ] = outputs[1]
    print(outputs[1])

    return outputs[1] # use val accuracy to search for best hyper-param set


# start hyper-param tuning
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 8)

final_prompt = max(best_prompt_per_round, key=best_prompt_per_round.get)

num_prompts = study.best_params['num of prompts']
cos_sim = study.best_params['Mean Cosine Sim among prompts']
lexical_sim = study.best_params['Mean Meteor among prompts']

## Performance Evaluation

In [ ]:
tuning_size = 200

print(final_prompt)
print()

# Regular expression to split the text into individual algorithms
algorithm_pattern = r"###------- Step \d: [\w\s]+"

# Split the text based on the pattern
split_text = re.split(algorithm_pattern, final_prompt)

# Extract the algorithm headers (for identification)
headers = re.findall(algorithm_pattern, final_prompt)

# Removing the first empty string from the split if exists (because of the split at the start)
split_text = [t.strip() for t in split_text if t.strip()]

# Create a dictionary where each algorithm is stored separately
algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

# Display each algorithm separately
steps = []
for header, content in algorithms.items():
    steps.append( [header, content] )
    # print(f"{header}:\n{content}\n")

total_right = 0
ADO_total_right = 0
total = 0

for i in range( tuning_size, len(data) ):
    question = data[i]['question']
    target = re.findall(r'\n#### (.*)', data[i]['answer'])[0]

    original_prompt = (
        'Given a math question, please solve it by returning the final answer as a number.' +
        f'\n\nQuestion: {question}' +
        "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
    )

    completion = client.chat.completions.create(
        model = model_id, temperature = 0, seed = 0,

        messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                    {"role": "user", "content": original_prompt}],
        timeout = 1200)

    ori_answer = completion.choices[0].message.content

    total += 1
    match = re.search(r'[-+]?\d*\.\d+|\d+', ori_answer)
    if match:
        ori_answer = float(match.group())

    if float(target) == float(ori_answer):
        total_right += 1


    overall_with_steps = "Original question statement: " + question + '\n\n'
    for i in range(len(algorithms)):

        reformulation_prompt = f"Please thoroughly reformulate the math question statement based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nQuestion statement to reformulate: {overall_with_steps}."

        completion = client.chat.completions.create(
                model = model_id, temperature = 0.0, max_tokens = 768,

                messages=[{"role": "system", "content": "Please reformulate and enrich the math question to be extremely informative and detailed for LLM to interpret better. You are allowed to infer and fill-in unspecified information based on your domain expertise!"},
                            {"role": "user", "content": reformulation_prompt}],
                timeout = 1200)

        reformulated_history = completion.choices[0].message.content
        overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'

    ADO_prompt = (
        'Given a math question, please solve it by returning the final answer as a number.' +
        f'\n\nQuestion: {overall_with_steps}' +
        "Output format: directly return an algebric number representing the answer; do NOT explain anything. You MUST follow this format: 'The answer is: 34.5'!!!"
    )

    completion = client.chat.completions.create(
        model = model_id, temperature = 0, seed = 0,

        messages=[{"role": "system", "content": 'Please solve the math question presented by outputting the final answer.'},
                    {"role": "user", "content": ADO_prompt}],
        timeout = 1200)

    answer = completion.choices[0].message.content

    match = re.search(r'[-+]?\d*\.\d+|\d+', answer)
    if match:
        answer = float(match.group())

    if float(target) == float(answer):
        ADO_total_right += 1

    print(target, ori_answer, answer)
    print(f'Total: {total}, Total Right: {total_right}, ADE Total Right: {ADO_total_right}')
    print('----------------------------------------------------------------------------------------')
    print()